In [ ]:
# @title




# # ===============================
# #   PART 0 — Install libraries
# # ===============================
# !pip install numpy pandas scikit-learn tensorflow matplotlib seaborn

# # ===============================
# #   PART 1 — Load NSL-KDD
# # ===============================
# import pandas as pd
# import numpy as np
# import tensorflow as tf
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, classification_report
# from tensorflow.keras import layers, models
# # URLs of NSL-KDD files
# train_url = "https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain+.txt"
# test_url  = "https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTest+.txt"

# cols = [
#     'duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent',
#     'hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted','num_root',
#     'num_file_creations','num_shells','num_access_files','num_outbound_cmds','is_host_login',
#     'is_guest_login','count','srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate',
#     'same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count','dst_host_srv_count',
#     'dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate',
#     'dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate',
#     'dst_host_srv_rerror_rate','label', 'difficulty_score' # Added 'difficulty_score' to account for all 43 columns
# ]

# train = pd.read_csv(train_url, names=cols)
# test  = pd.read_csv(test_url, names=cols)

# # Drop the difficulty_score column as it's not a feature for the model
# train = train.drop(columns=['difficulty_score'])
# test = test.drop(columns=['difficulty_score'])

# # Convert labels to binary
# train['label'] = train['label'].apply(lambda x: 0 if x == 'normal' else 1)
# test['label']  = test['label'].apply(lambda x: 0 if x == 'normal' else 1)

# # One-hot encode categorical
# cat = ['protocol_type','service','flag']
# train = pd.get_dummies(train, columns=cat)
# test = pd.get_dummies(test, columns=cat)

# # Align columns so train and test have the same set of features
# train_encoded, test_encoded = train.align(test, join='left', axis=1, fill_value=0)

# # Now both DataFrames have identical columns
# print("Train shape:", train_encoded.shape)
# print("Test shape:", test_encoded.shape)

# # test all numeric or not
# #print(train_encoded.select_dtypes(include=['object']).head())
# #print(test_encoded.select_dtypes(include=['object']).head())

# # divide data to xtrain and ytrain
# X_train = train_encoded.drop('label', axis=1).astype(float)
# y_train = train_encoded['label']

# X_test = test_encoded.drop('label', axis=1).astype(float)
# y_test = test_encoded['label']

# # Now both DataFrames have identical columns
# print("Train shape:", train_encoded.shape)
# print("Test shape:", test_encoded.shape)

# # ===============================
# #   PART 2 — Build IDS model (MLP)
# # ===============================

# # def build_ids(input_dim):
# #     model = models.Sequential([
# #         layers.Dense(64, activation='relu', input_shape=(input_dim,)),
# #         layers.Dense(32, activation='relu'),
# #         layers.Dense(1, activation='sigmoid')
# #     ])
# #     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# #     return model

# # ids = build_ids(X_train.shape[1])
# # ids.fit(X_train, y_train, epochs=5, batch_size=256, validation_split=0.1)
# # loss, accuracy = ids.evaluate(X_test, y_test)
# # print(f"Test accuracy: {accuracy}")


# # ----------------------------
# # 1. Reshape input for LSTM
# # ----------------------------
# # LSTM expects: (batch, timesteps, features)
# # We treat each sample as 1 timestep with all features
# X_train_lstm = np.expand_dims(X_train, axis=1)   # shape becomes (N, 1, features)
# X_test_lstm  = np.expand_dims(X_test, axis=1)

# # ----------------------------
# # 2. Build LSTM-based IDS
# # ----------------------------
# def build_ids_lstm(input_dim,lr=0.001):
#     model = models.Sequential([
#         layers.LSTM(64, input_shape=(1, input_dim), return_sequences=False),
#         layers.Dense(32, activation='relu'),
#         layers.Dropout(0.3),
#         layers.Dense(1, activation='sigmoid')
#     ])

#     model.compile(  optimizer=tf.keras.optimizers.Adam(lr),
#                   loss='binary_crossentropy',
#                   metrics=['accuracy'])
#     return model

# ids = build_ids_lstm(X_train.shape[1])

# # ----------------------------
# # 3. Train the model
# # ----------------------------
# ids.fit(X_train_lstm, y_train,
#         epochs=50,
#         batch_size=256,
#         validation_split=0.1,
#         verbose=1)

# # ----------------------------
# # 4. Evaluate
# # ----------------------------
# loss, accuracy = ids.evaluate(X_test_lstm, y_test)
# print(f"Test accuracy: {accuracy}")

# # ===============================
# #   PART ----- —  an other way to Generate Adversarial Examples (FGSM) using problem space
# # ===============================
# # ===============================
# # Dhole Algorithm (2025)
# # ===============================

# def fitness_function(mask):
#     """
#     mask: binary vector (0/1) length = number of features
#     """
#     # avoid empty feature set
#     if np.sum(mask) == 0:
#         return np.inf

#     selected_idx = np.where(mask == 1)[0]

#     X_sel = X_train.iloc[:, selected_idx]
#     X_val_sel = X_test.iloc[:, selected_idx]

#     # simple IDS model (fast evaluation)
#     model = LogisticRegression(max_iter=200)
#     model.fit(X_sel, y_train)

#     y_pred = model.predict(X_val_sel)
#     acc = accuracy_score(y_test, y_pred)

#     # penalty for using too many features
#     penalty = 0.01 * np.sum(mask)

#     return -(acc - penalty)   # minimization


# import numpy as np

# def dhole_opt(N=20, T=5):
#     dim = X_train.shape[1]

#     # initialize population (binary masks)
#     X = np.random.randint(0, 2, size=(N, dim))

#     fitness = np.array([fitness_function(ind) for ind in X])

#     best_idx = np.argmin(fitness)
#     prey_global = X[best_idx].copy()
#     best_fit = fitness[best_idx]

#     curve = []

#     for t in range(T):
#         C = 1 - t / T

#         for i in range(N):
#             Xnew = X[i].copy()

#             if np.random.rand() < 0.5:
#                 # exploration
#                 j = np.random.randint(dim)
#                 Xnew[j] = 1 - Xnew[j]
#             else:
#                 # exploitation (move toward best)
#                 diff = prey_global ^ X[i]
#                 flip = np.random.rand(dim) < (C * diff)
#                 Xnew[flip] = prey_global[flip]

#             new_fit = fitness_function(Xnew)

#             if new_fit < fitness[i]:
#                 X[i] = Xnew
#                 fitness[i] = new_fit

#                 if new_fit < best_fit:
#                     best_fit = new_fit
#                     prey_global = Xnew.copy()

#         curve.append(best_fit)

#     return prey_global, best_fit, curve


# best_mask, best_score, curve = dhole_opt()

# selected_features = X_train.columns[best_mask == 1]

# print("Selected features:", list(selected_features))
# print("Number of features:", len(selected_features))


# model_opt = build_ids_lstm(len(selected_features))
# model_opt.fit(
#     X_train.iloc[:, selected_features], # Corrected indexing
#     y_train,
#     epochs=50,
#     batch_size=265
# )

# opt_acc = model_opt.evaluate(
#     X_test.iloc[:, selected_features], y_test # Corrected indexing
# )[1]

# print("Optimized Accuracy:", opt_acc)


# # ===============================
# #   PART 3 — Generate Adversarial Examples (FGSM) using feature space
# # ===============================

# # FGSM attack
# # def fgsm_attack(model, x, y, eps=0.1):
# #     x_tensor = tf.convert_to_tensor(x, dtype=tf.float32)
# #     y_tensor = tf.convert_to_tensor(y, dtype=tf.float32)

# #     # Reshape y_tensor to match the output shape of the model (batch_size, 1)
# #     y_tensor = tf.expand_dims(y_tensor, axis=-1)

# #     with tf.GradientTape() as tape:
# #         tape.watch(x_tensor)
# #         pred = model(x_tensor)
# #         loss = tf.keras.losses.binary_crossentropy(y_tensor, pred)

# #     grad = tape.gradient(loss, x_tensor)
# #     ae = x_tensor + eps * tf.sign(grad)
# #     return np.clip(ae.numpy(), 0, 1)

# # # Generate 5k adversarial samples
# # X_adv = fgsm_attack(ids, X_train[:5000], y_train[:5000])
# # y_adv = y_train[:5000].copy()
# # #show which features affected by FGSM
# # diff = X_adv - X_train[:5000]
# # changed_features = (diff != 0).sum(axis=0)
# # print(changed_features)

# # ===============================
# #   PART 3 — Generate Adversarial Examples (FGSM) using problem space
# # ===============================
# # ========== 1) Features allowed to be modified ==========
# # modifiable_features = [
# #     'duration','src_bytes','srv_count',
# #     'count','dst_host_count','dst_host_srv_count'
# # ]

# # p = 0.75 # = 5% max change per feature (problem space)

# # # copy of data before encoding
# # raw_test = pd.read_csv(test_url, names=cols)
# # raw_test = raw_test.drop(columns=['difficulty_score'])
# # raw_test['label'] = raw_test['label'].apply(lambda x: 0 if x == 'normal' else 1)

# # # ========== 2) FGSM on encoded space but apply to raw features ==========
# # def problem_space_fgsm(model, X_raw, X_encoded, y, eps=0.1):

# #     # convert int tensor
# #     X_raw = X_raw.copy()
# #     X = tf.convert_to_tensor(X_encoded, dtype=tf.float32)
# #     y = tf.convert_to_tensor(y, dtype=tf.float32)
# #     y = tf.expand_dims(y, axis=-1)

# #     with tf.GradientTape() as tape:
# #         tape.watch(X)
# #         pred = model(X)
# #         loss = tf.keras.losses.binary_crossentropy(y, pred)

# #     grad = tape.gradient(loss, X)

# #     # identify gradient of features
# #     grad_sign = np.sign(grad)

# #     # ========== 3) modify only modifiable features  ==========
# #     for f in modifiable_features:
# #         g = grad_sign[:, X_test.columns.get_loc(f)]

# #         # أقصى تغير مسموح به في problem-space p%
# #         delta = p * X_raw[f].abs()

# #         # القيمة الجديدة
# #         X_raw[f] = X_raw[f] + g * delta

# #         # ضمان عدم السالب
# #         X_raw[f] = np.clip(X_raw[f], 0, None)

# #     # ========== 4) apply One-hot ==========
# #     adv = pd.get_dummies(X_raw, columns=['protocol_type','service','flag'])

# #     # align مع train
# #     adv_encoded = adv.reindex(columns=test_encoded.columns, fill_value=0)

# #     # فصل X و y
# #     X_adv = adv_encoded.drop('label', axis=1).astype(float)
# #     y_adv = adv_encoded['label']

# #     return X_adv.values, y_adv.values


# # # # Generate 5k adversarial samples
# # X_raw_subset = raw_test.iloc[:5000].reset_index(drop=True)
# # X_encoded_subset = X_test.iloc[:5000].values
# # y_subset = y_test.iloc[:5000].values

# # # Generate adversarial samples for ALL test data
# # # X_raw_subset = raw_test.reset_index(drop=True)
# # # X_encoded_subset = X_test.values
# # # y_subset = y_test.values

# # X_adv, y_adv = problem_space_fgsm(ids, X_raw_subset, X_encoded_subset, y_subset, eps=0.1)

# # print("Generated adversarial examples (problem space):", X_adv.shape)

# # #show which features affected by FGSM
# # diff = X_adv - X_test[:5000]
# # changed_features = (diff != 0).sum(axis=0)
# # print(changed_features)



# # ===============================
# #   PART 3 —  an other way to Generate Adversarial Examples (FGSM) using problem space
# # ===============================
# # ===============================
# # FGSM EXACTLY LIKE MANDA (2024)
# # ===============================



# # -------- 1) Problem-space numeric features only --------
# numeric_features = [
#     'duration','src_bytes','count','srv_count','dst_host_count','dst_host_srv_count'
# ]

# # -------- 2) All categorical that must NOT change --------
# categorical_features = ['protocol_type','service','flag']
# # copy of data before encoding
# raw_test = pd.read_csv(test_url, names=cols)
# raw_test = raw_test.drop(columns=['difficulty_score'])
# raw_test['label'] = raw_test['label'].apply(lambda x: 0 if x == 'normal' else 1)
# X_raw_41 = raw_test.drop('label', axis=1)
# # X_encoded_121 = pd.get_dummies(X_raw_41, columns=categorical_features)
# # X_train_encoded_cols = X_encoded_121.columns

# def manda_fgsm(model, X_raw_unencoded, X_encoded_aligned, y_labels, eps=0.1, model_expected_cols=None,p=0.05):

#     # ---------- Step 1: FGSM in feature-space ----------
#     X = tf.convert_to_tensor(X_encoded_aligned, dtype=tf.float32)
#     X = tf.expand_dims(X, axis=1) # Add the timesteps dimension for LSTM
#     y_tensor = tf.convert_to_tensor(y_labels, dtype=tf.float32)
#     y_tensor = tf.expand_dims(y_tensor, axis=-1)

#     with tf.GradientTape() as tape:
#         tape.watch(X)
#         pred = model(X)
#         loss = tf.keras.losses.binary_crossentropy(y_tensor, pred)

#     grad = tape.gradient(loss, X).numpy()
#     grad_sign = np.sign(grad)

#     # Ensure model_expected_cols is provided
#     if model_expected_cols is None:
#         raise ValueError("model_expected_cols must be provided for problem-space FGSM")

#     # Nullify perturbations on categorical / non-diff features
#     # Iterate through categorical features and find their one-hot encoded columns in model_expected_cols
#     for cat in categorical_features:
#         # Find indices of one-hot encoded columns corresponding to the categorical feature
#         col_indices = [i for i, c in enumerate(model_expected_cols) if c.startswith(cat+'_')]
#         if col_indices: # Only modify if such columns exist
#             grad_sign[:, 0, col_indices] = 0 # Adjust index for 3D grad_sign

#     # ---------- Step 2: Map back to problem-space ----------
#     X_raw_adv = X_raw_unencoded.copy()

#     for f in numeric_features:
#         # Check if the numeric feature exists in the model's expected columns
#         if f in model_expected_cols:
#             # Apply perturbation to the raw feature based on the gradient of its encoded counterpart
#             # The gradient sign for 'f' is at col_index_in_encoded_aligned in grad_sign
#             idx = model_expected_cols.get_loc(f)
#             g = grad_sign[:, 0, idx]               # FGSM direction (+1 / -1) - Adjusted for 3D grad_sign

#             delta = p * X_raw_adv[f].abs()     # allowed change = p%

#             # apply modification
#             X_raw_adv[f] = X_raw_adv[f] +  (eps * g * delta)
#             # X_raw_adv[f] = X_raw_adv[f] + eps * grad_sign[:, idx] * np.abs(X_raw_unencoded[f])
#             X_raw_adv[f] = np.clip(X_raw_adv[f], 0, None) # Ensure non-negative

#     # Re-encode categorical features from the modified raw data
#     adv = pd.get_dummies(X_raw_adv, columns=categorical_features)

#     # Align with model's expected columns (X_train.columns)
#     X_adv_encoded = adv.reindex(columns=model_expected_cols, fill_value=0)

#     # Separate X and y (y labels remain unchanged, as it's an adversarial attack on X)
#     X_final = X_adv_encoded.astype(float).values # Removed .drop('label', axis=1)
#     y_final = y_labels # Labels remain the same

#     return X_final, y_final


# # # # Generate 5k adversarial samples
# X_raw_subset = X_raw_41.iloc[:200].reset_index(drop=True)
# X_encoded_subset = X_test.iloc[:200].values # Use the correctly aligned X_test data
# y_subset = y_test.iloc[:200].values

# X_adv, y_adv = manda_fgsm(model_opt,X_test.iloc[:, selected_features], X_encoded_subset, y_subset, eps=0.1, model_expected_cols=X_train.columns,p=0.075) # Pass X_train.columns for alignment # Corrected indexing

# print("Generated adversarial examples (problem space):", X_adv.shape)

# #show which features affected by FGSM
# diff = pd.DataFrame(X_adv, columns=X_train.columns) - X_test.iloc[:200] # Convert X_adv to DataFrame with correct columns for comparison
# changed_features = (diff != 0).sum(axis=0)
# print(changed_features)

# # ===============================
# #   PART 4 — Compute MANIFOLD SCORE
# # ===============================

# # PCA projection distance as manifold score
# from sklearn.decomposition import PCA
# from sklearn.metrics import pairwise_distances

# pca = PCA(n_components=10)
# pca.fit(X_train)

# def manifold_score(x):
#     proj = pca.inverse_transform(pca.transform(x))
#     return np.mean((x - proj)**2, axis=1)

# manifold_clean = manifold_score(X_train[:200])
# manifold_adv   = manifold_score(X_adv)


# # ===============================
# #   PART 5 — Compute DB SCORE (Deep Boundary)
# # ===============================

# def db_score(model, x):
#     # Reshape input for the LSTM model
#     x_reshaped = np.expand_dims(x, axis=1)
#     with tf.GradientTape() as tape:
#         x_t = tf.convert_to_tensor(x_reshaped, dtype=tf.float32)
#         tape.watch(x_t)
#         pred = model(x_t)
#     grad = tape.gradient(pred, x_t).numpy()
#     # The gradient will also be 3D (batch, 1, features), so squeeze the middle dimension
#     return np.mean(np.abs(grad[:, 0, :]), axis=1)

# db_clean = db_score(ids, X_train[:200])
# db_adv   = db_score(ids, X_adv)


# # ===============================
# #   PART 6 — Build MANDA dataset
# # ===============================

# S1 = np.concatenate([manifold_clean, manifold_adv])
# S2 = np.concatenate([db_clean, db_adv])
# Y  = np.concatenate([np.zeros_like(manifold_clean), np.ones_like(manifold_adv)])

# df_manda = pd.DataFrame({'manifold': S1, 'db': S2, 'label': Y})
# df_manda.head()
# #print(df_manda)

# # ===============================
# #   PART 7 — Train MANDA (Logistic Regression)
# # ===============================

# from sklearn.metrics import accuracy_score, roc_auc_score

# clf = LogisticRegression()
# clf.fit(df_manda[['manifold', 'db']], df_manda['label'])

# pred = clf.predict(df_manda[['manifold', 'db']])
# print("MANDA accuracy:", accuracy_score(df_manda['label'], pred))
# print("AUC:", roc_auc_score(df_manda['label'], pred))

# # ===============================
# #   PART 8 — Fixing FPR at 5% or 15%
# # ===============================

# from sklearn.metrics import roc_curve

# scores = clf.predict_proba(df_manda[['manifold', 'db']])[:,1]
# fpr, tpr, th = roc_curve(df_manda['label'], scores)

# def get_threshold(target_fpr):
#     idx = np.argmin(np.abs(fpr - target_fpr))
#     return th[idx]

# thr_5  = get_threshold(0.05)
# thr_15 = get_threshold(0.15)

# print("Threshold at 5% FPR:", thr_5)
# print("Threshold at 15% FPR:", thr_15)


In [1]:
# ===============================
#   PART 0 — Install libraries
# ===============================
!pip install numpy pandas scikit-learn tensorflow matplotlib seaborn
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.preprocessing import LabelEncoder


In [2]:

# ===============================
#   PART 1 — Load NSL-KDD
# ===============================
# URLs of NSL-KDD files
train_url = "https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain+.txt"
test_url  = "https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTest+.txt"

cols = [
    'duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent',
    'hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted','num_root',
    'num_file_creations','num_shells','num_access_files','num_outbound_cmds','is_host_login',
    'is_guest_login','count','srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate',
    'same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count','dst_host_srv_count',
    'dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate',
    'dst_host_srv_rerror_rate','label', 'difficulty_score' # Added 'difficulty_score' to account for all 43 columns
]

train = pd.read_csv(train_url, names=cols)
test  = pd.read_csv(test_url, names=cols)

# Drop the difficulty_score column as it's not a feature for the model
train = train.drop(columns=['difficulty_score'])
test = test.drop(columns=['difficulty_score'])

# Convert labels to binary
train['label'] = train['label'].apply(lambda x: 0 if x == 'normal' else 1)
test['label']  = test['label'].apply(lambda x: 0 if x == 'normal' else 1)

# Catogrical features
cat = ['protocol_type','service','flag']

# # One-hot encode categorical
# train = pd.get_dummies(train, columns=cat)
# test = pd.get_dummies(test, columns=cat)

# label encoding
encoders = {}

for col in cat:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col]  = le.transform(test[col])
    encoders[col] = le

# Align columns so train and test have the same set of features
train_encoded, test_encoded = train.align(test, join='left', axis=1, fill_value=0)

# Now both DataFrames have identical columns
print("Train shape:", train_encoded.shape)
print("Test shape:", test_encoded.shape)

# test all numeric or not
#print(train_encoded.select_dtypes(include=['object']).head())
#print(test_encoded.select_dtypes(include=['object']).head())

# divide data to xtrain and ytrain
X_train = train_encoded.drop('label', axis=1).astype(float)
y_train = train_encoded['label']

X_test = test_encoded.drop('label', axis=1).astype(float)
y_test = test_encoded['label']

# Now both DataFrames have identical columns
print("Train shape:", train_encoded.shape)
print("Test shape:", test_encoded.shape)

Train shape: (125973, 42)
Test shape: (22544, 42)
Train shape: (125973, 42)
Test shape: (22544, 42)


In [32]:
# ===============================
#   PART 2 — Build IDS model (MLP)
# ===============================

# def build_ids(input_dim):
#     model = models.Sequential([
#         layers.Dense(64, activation='relu', input_shape=(input_dim,)),
#         layers.Dense(32, activation='relu'),
#         layers.Dense(1, activation='sigmoid')
#     ])
#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#     return model

# ids = build_ids(X_train.shape[1])
# ids.fit(X_train, y_train, epochs=5, batch_size=256, validation_split=0.1)
# loss, accuracy = ids.evaluate(X_test, y_test)
# print(f"Test accuracy: {accuracy}")




# ===============================
#   PART 2 — Build IDS model (LSTM)
# ===============================

# ----------------------------
# 1. Reshape input for LSTM
# ----------------------------
# LSTM expects: (batch, timesteps, features)
# We treat each sample as 1 timestep with all features
X_train_lstm = np.expand_dims(X_train, axis=1)   # shape becomes (N, 1, features)
X_test_lstm  = np.expand_dims(X_test, axis=1)

#add function to change timestep >1
def make_lstm_sequences(X, y, timesteps=5):
    X_seq, y_seq = [], []

    for i in range(len(X) - timesteps + 1):
        X_seq.append(X[i:i+timesteps])
        y_seq.append(y[i+timesteps-1])  # label آخر خطوة

    return np.array(X_seq), np.array(y_seq)

X_train_lstm, y_train_lstm = make_lstm_sequences(
    X_train.values, y_train.values, timesteps=5
)

X_test_lstm, y_test_lstm = make_lstm_sequences(
    X_test.values, y_test.values, timesteps=5
)

print(X_train_lstm.shape)  # (N-4, 5, features)
print(X_test_lstm.shape)



# ----------------------------
# 2. Build LSTM-based IDS
# ----------------------------
def build_ids_lstm(input_dim,lr=0.001):
    model = models.Sequential([
        layers.LSTM(128, input_shape=(1, input_dim), return_sequences=True),
        layers.Dropout(0.3),

        layers.LSTM(64, return_sequences=False),
        layers.BatchNormalization(),

        layers.Dense(32, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(  optimizer=tf.keras.optimizers.Adam(lr),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

ids = build_ids_lstm(X_train.shape[1])

# ----------------------------
# 3. Train the model
# ----------------------------
ids.fit(X_train_lstm, y_train_lstm,
        epochs=5,
        batch_size=256,
        validation_split=0.1,
        verbose=1)

# ----------------------------
# 4. Evaluate
# ----------------------------
loss, accuracy = ids.evaluate(X_test_lstm, y_test_lstm)
print("Test shape:", X_test_lstm.shape)
print(f"Test accuracy: {accuracy}")


(125969, 5, 41)
(22540, 5, 41)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
443/443 ━━━━━━━━━━━━━━━━━━━━ 29s 53ms/step - accuracy: 0.8733 - loss: 0.2761 - val_accuracy: 0.9702 - val_loss: 0.0732
Epoch 2/5
443/443 ━━━━━━━━━━━━━━━━━━━━ 22s 50ms/step - accuracy: 0.9682 - loss: 0.0852 - val_accuracy: 0.9809 - val_loss: 0.0588
Epoch 3/5
443/443 ━━━━━━━━━━━━━━━━━━━━ 23s 53ms/step - accuracy: 0.9730 - loss: 0.0733 - val_accuracy: 0.9751 - val_loss: 0.0593
Epoch 4/5
443/443 ━━━━━━━━━━━━━━━━━━━━ 24s 53ms/step - accuracy: 0.9734 - loss: 0.0680 - val_accuracy: 0.9749 - val_loss: 0.0606
Epoch 5/5
443/443 ━━━━━━━━━━━━━━━━━━━━ 23s 52ms/step - accuracy: 0.9741 - loss: 0.0673 - val_accuracy: 0.9799 - val_loss: 0.0530


ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 22540
'y' sizes: 22544


In [28]:
#perform attack on all features


# ===============================
#   PART 3 —  an other way to Generate Adversarial Examples (FGSM) using problem space
# ===============================
# ===============================
# FGSM EXACTLY LIKE MANDA (2024)
# ===============================
# -------- 1) Problem-space all numeric features only --------
categorical_features = ['protocol_type','service','flag']
numeric_features = [
    col for col in X_test.columns
    if  col not in categorical_features
]

# -------- 2) All categorical that must NOT change --------
# copy of data before encoding
# raw_test = pd.read_csv(test_url, names=cols)
# raw_test = raw_test.drop(columns=['difficulty_score'])
# raw_test['label'] = raw_test['label'].apply(lambda x: 0 if x == 'normal' else 1)
# X_raw_41 = raw_test.drop('label', axis=1)
# X_encoded_121 = pd.get_dummies(X_raw_41, columns=categorical_features)
# X_train_encoded_cols = X_encoded_121.columns

def manda_fgsm(model, X_raw_unencoded_full, X_encoded_aligned_for_model, y_labels, eps=0.1, full_model_cols=None, subset_model_cols=None, p=0.05):

    # ---------- Step 1: FGSM in feature-space (on subset features) ----------
    X_model_input_tensor = tf.convert_to_tensor(X_encoded_aligned_for_model, dtype=tf.float32)
    X_model_input_tensor = tf.expand_dims(X_model_input_tensor, axis=1) # Add the timesteps dimension for LSTM
    y_tensor = tf.convert_to_tensor(y_labels, dtype=tf.float32)
    y_tensor = tf.expand_dims(y_tensor, axis=-1)

    with tf.GradientTape() as tape:
        tape.watch(X_model_input_tensor)
        pred = model(X_model_input_tensor)
        loss = tf.keras.losses.binary_crossentropy(y_tensor, pred)

    grad = tape.gradient(loss, X_model_input_tensor).numpy()
    grad_sign = np.sign(grad) # grad_sign will have shape (batch_size, 1, len(subset_model_cols))

    if full_model_cols is None or subset_model_cols is None:
        raise ValueError("full_model_cols and subset_model_cols must be provided for problem-space FGSM")

    # Nullify perturbations on categorical / non-diff features within the subset
    for cat in categorical_features:
        # Find indices of one-hot encoded columns corresponding to the categorical feature within subset_model_cols
        col_indices_in_subset = [i for i, c in enumerate(subset_model_cols) if c.startswith(cat+'_')]
        if col_indices_in_subset:
            grad_sign[:, 0, col_indices_in_subset] = 0 # This now correctly indexes into the subset grad_sign

    # ---------- Step 2: Map back to problem-space ----------
    X_raw_adv = X_raw_unencoded_full.copy() # Operate on the full raw DataFrame

    for f in numeric_features:
        # Check if the numeric feature exists in the subset_model_cols (because grad_sign is based on it)
        if f in subset_model_cols:
            idx_in_subset = subset_model_cols.get_loc(f)
            g = grad_sign[:, 0, idx_in_subset] # FGSM direction (+1 / -1)

            delta = p * X_raw_adv[f].abs()     # allowed change = p%

            # apply modification
            X_raw_adv[f] = X_raw_adv[f] +  (eps * g * delta)
            X_raw_adv[f] = np.clip(X_raw_adv[f], 0, None) # Ensure non-negative

    # Re-encode categorical features from the modified raw data
    adv = pd.get_dummies(X_raw_adv, columns=categorical_features)

    # Align with the full model's expected columns (X_train.columns) for the final output
    X_adv_encoded = adv.reindex(columns=full_model_cols, fill_value=0)

    # Separate X and y
    X_final = X_adv_encoded.astype(float).values
    y_final = y_labels

    return X_final, y_final


# # # Generate 5k adversarial samples (using 200 samples for consistency)
X_raw_subset_full = X_test.reset_index(drop=True)
# Filter X_test for selected features and consistent row count
X_test_encoded_for_model = X_test.values
y_subset = y_test.values

# Diagnostic prints to verify shapes before calling manda_fgsm
print(f"Shape of X_raw_subset_full: {X_raw_subset_full.shape}")
print(f"Shape of X_test_encoded_for_model: {X_test_encoded_for_model.shape}")
print(f"Shape of y_subset: {y_subset.shape}")

X_adv, y_adv = manda_fgsm(
    ids,
    X_raw_subset_full,
    X_test_encoded_for_model,
    y_subset,
    eps=0.2,
    full_model_cols=X_train.columns,
    subset_model_cols=X_train.columns,
    p=0.1
)

print("Generated adversarial examples (problem space):", X_adv.shape)

#show which features affected by FGSM
diff = pd.DataFrame(X_adv, columns=X_train.columns) - X_test # Convert X_adv to DataFrame with correct columns for comparison
changed_features = (diff[numeric_features] != 0).sum(axis=0)
print(changed_features)



Shape of X_raw_subset_full: (22544, 41)
Shape of X_test_encoded_for_model: (22544, 41)
Shape of y_subset: (22544,)
Generated adversarial examples (problem space): (22544, 41)
duration                        2879
src_bytes                      13760
dst_bytes                      12559
land                               7
wrong_fragment                   100
urgent                             7
hot                              950
num_failed_logins                476
logged_in                       9058
num_compromised                  350
root_shell                        39
su_attempted                       2
num_root                          32
num_file_creations                36
num_shells                        16
num_access_files                  64
num_outbound_cmds                  0
is_host_login                     11
is_guest_login                   641
count                          21388
srv_count                      21376
serror_rate                     3158
srv_serror_

In [29]:
# training data after attack on all numeric feaures using LSTM

#merge clean and adversarial data together
X_train_adv = np.vstack([X_train.values, X_adv])
y_train_adv = np.concatenate([y_train.values, y_adv])

#add lstm dimesion
X_train_adv_lstm = np.expand_dims(X_train_adv, axis=1)
X_test_lstm = np.expand_dims(X_test.values, axis=1)

#train LSTM

fgsm= build_ids_lstm(X_train_adv.shape[1])
history = fgsm.fit(
    X_train_adv_lstm,
    y_train_adv,
    epochs=5,
    batch_size=256,
    validation_split=0.2,
    verbose=1
)
#accuracy on clean data
loss_clean, acc_clean = fgsm.evaluate(X_test_lstm, y_test.values)
print("Accuracy on clean test:", acc_clean)

#accuracy on adversarial data
X_adv_lstm = np.expand_dims(X_adv, axis=1)
loss_adv, acc_adv = fgsm.evaluate(X_adv_lstm, y_adv)
print("Accuracy on adversarial test:", acc_adv)

Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


465/465 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9285 - loss: 0.2008 - val_accuracy: 0.8070 - val_loss: 0.7322
Epoch 2/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9751 - loss: 0.0626 - val_accuracy: 0.8102 - val_loss: 0.9022
Epoch 3/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9849 - loss: 0.0480 - val_accuracy: 0.7992 - val_loss: 0.9566
Epoch 4/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9832 - loss: 0.0464 - val_accuracy: 0.7981 - val_loss: 0.9054
Epoch 5/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9824 - loss: 0.0462 - val_accuracy: 0.7931 - val_loss: 1.0372
705/705 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7692 - loss: 1.0373
Accuracy on clean test: 0.7665010690689087
705/705 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7372 - loss: 1.3218
Accuracy on adversarial test: 0.7325230836868286


In [ ]:
# Selected features by Dhole


def fitness_function(mask):
    """
    mask: binary vector (0/1) length = number of features
    """
    # avoid empty feature set
    if np.sum(mask) == 0:
        return np.inf

    selected_idx = np.where(mask == 1)[0]

    X_sel = X_train.iloc[:, selected_idx]
    X_val_sel = X_test.iloc[:, selected_idx]

    # simple IDS model (fast evaluation)
    model = LogisticRegression(max_iter=2)
    model.fit(X_sel, y_train)

    y_pred = model.predict(X_val_sel)
    acc = accuracy_score(y_test, y_pred)

    # penalty for using too many features
    penalty = 0.01 * np.sum(mask)

    return -(acc - penalty)   # minimization

def dhole_opt(N=20, T=5):
    dim = X_train.shape[1]

    # initialize population (binary masks)
    X = np.random.randint(0, 2, size=(N, dim))

    fitness = np.array([fitness_function(ind) for ind in X])

    best_idx = np.argmin(fitness)
    prey_global = X[best_idx].copy()
    best_fit = fitness[best_idx]

    curve = []

    for t in range(T):
        C = 1 - t / T

        for i in range(N):
            Xnew = X[i].copy()

            if np.random.rand() < 0.5:
                # exploration
                j = np.random.randint(dim)
                Xnew[j] = 1 - Xnew[j]
            else:
                # exploitation (move toward best)
                diff = prey_global ^ X[i]
                flip = np.random.rand(dim) < (C * diff)
                Xnew[flip] = prey_global[flip]

            new_fit = fitness_function(Xnew)

            if new_fit < fitness[i]:
                X[i] = Xnew
                fitness[i] = new_fit

                if new_fit < best_fit:
                    best_fit = new_fit
                    prey_global = Xnew.copy()

        curve.append(best_fit)

    return prey_global, best_fit, curve

best_mask, best_score, curve = dhole_opt()

selected_features = X_train.columns[best_mask == 1]

print("Selected features:", list(selected_features))
print("Number of features:", len(selected_features))

#test LSTM accuracy after selection features
model_opt = build_ids_lstm(len(selected_features))
X_train_sel = X_train.loc[:, selected_features]
X_test_sel  = X_test.loc[:, selected_features]
X_train_sel_lstm = np.expand_dims(X_train_sel.values, axis=1)
X_test_sel_lstm  = np.expand_dims(X_test_sel.values, axis=1)

model_opt.fit(
    X_train_sel_lstm,
    y_train,
    epochs=5,
    batch_size=256,
    validation_split=0.1,
    verbose=1
)

opt_acc = model_opt.evaluate(
          X_test_sel_lstm, y_test # Corrected indexing
)[1]
print("Test shape:", X_test_sel_lstm.shape)
print("Train shape:", X_train_sel_lstm.shape)
print("Optimized Accuracy:", opt_acc)

# predictions
y_pred_prob = model_opt.predict(X_test_sel_lstm).ravel()
y_pred = (y_pred_prob > 0.5).astype(int)

print("MANDA LSTM Accuracy:", accuracy_score(y_test, y_pred))
print("MANDA LSTM AUC:", roc_auc_score(y_test, y_pred_prob))

In [ ]:
# ===============================
#   PART 3 —  an other way to Generate Adversarial Examples (FGSM) using problem space
# ===============================
# ===============================
# FGSM EXACTLY LIKE MANDA (2024)
# ===============================



# -------- 1) Problem-space numeric features only --------
categorical_features = ['protocol_type','service','flag']
numeric_features = [
    f for f in selected_features
    if f not in categorical_features
]

# -------- 2) All categorical that must NOT change --------
# copy of data before encoding
# raw_test = pd.read_csv(test_url, names=cols)
# raw_test = raw_test.drop(columns=['difficulty_score'])
# raw_test['label'] = raw_test['label'].apply(lambda x: 0 if x == 'normal' else 1)
# X_raw_41 = raw_test.drop('label', axis=1)
# X_encoded_121 = pd.get_dummies(X_raw_41, columns=categorical_features)
# X_train_encoded_cols = X_encoded_121.columns

def manda_fgsm(model, X_raw_unencoded_full, X_encoded_aligned_for_model, y_labels, eps=0.1, full_model_cols=None, subset_model_cols=None, p=0.05):

    # ---------- Step 1: FGSM in feature-space (on subset features) ----------
    X_model_input_tensor = tf.convert_to_tensor(X_encoded_aligned_for_model, dtype=tf.float32)
    X_model_input_tensor = tf.expand_dims(X_model_input_tensor, axis=1) # Add the timesteps dimension for LSTM
    y_tensor = tf.convert_to_tensor(y_labels, dtype=tf.float32)
    y_tensor = tf.expand_dims(y_tensor, axis=-1)

    with tf.GradientTape() as tape:
        tape.watch(X_model_input_tensor)
        pred = model(X_model_input_tensor)
        loss = tf.keras.losses.binary_crossentropy(y_tensor, pred)

    grad = tape.gradient(loss, X_model_input_tensor).numpy()
    grad_sign = np.sign(grad) # grad_sign will have shape (batch_size, 1, len(subset_model_cols))

    if full_model_cols is None or subset_model_cols is None:
        raise ValueError("full_model_cols and subset_model_cols must be provided for problem-space FGSM")

    # Nullify perturbations on categorical / non-diff features within the subset
    for cat in categorical_features:
        # Find indices of one-hot encoded columns corresponding to the categorical feature within subset_model_cols
        col_indices_in_subset = [i for i, c in enumerate(subset_model_cols) if c.startswith(cat+'_')]
        if col_indices_in_subset:
            grad_sign[:, 0, col_indices_in_subset] = 0 # This now correctly indexes into the subset grad_sign

    # ---------- Step 2: Map back to problem-space ----------
    X_raw_adv = X_raw_unencoded_full.copy() # Operate on the full raw DataFrame

    for f in numeric_features:
        # Check if the numeric feature exists in the subset_model_cols (because grad_sign is based on it)
        if f in subset_model_cols:
            idx_in_subset = subset_model_cols.get_loc(f)
            g = grad_sign[:, 0, idx_in_subset] # FGSM direction (+1 / -1)

            delta = p * X_raw_adv[f].abs()     # allowed change = p%

            # apply modification
            X_raw_adv[f] = X_raw_adv[f] +  (eps * g * delta)
            X_raw_adv[f] = np.clip(X_raw_adv[f], 0, None) # Ensure non-negative

    # Re-encode categorical features from the modified raw data
    adv = pd.get_dummies(X_raw_adv, columns=categorical_features)

    # Align with the full model's expected columns (X_train.columns) for the final output
    X_adv_encoded = adv.reindex(columns=full_model_cols, fill_value=0)

    # Separate X and y
    X_final = X_adv_encoded.astype(float).values
    y_final = y_labels

    return X_final, y_final


# # # Generate 5k adversarial samples (using 200 samples for consistency)
X_raw_subset_full = X_test.reset_index(drop=True)
# Filter X_test for selected features and consistent row count
X_test_encoded_for_model = X_test.loc[:, selected_features].values
y_subset = y_test.values

# Diagnostic prints to verify shapes before calling manda_fgsm
print(f"Shape of X_raw_subset_full: {X_raw_subset_full.shape}")
print(f"Shape of X_test_encoded_for_model: {X_test_encoded_for_model.shape}")
print(f"Shape of y_subset: {y_subset.shape}")

X_adv, y_adv = manda_fgsm(
    model_opt,
    X_raw_subset_full,
    X_test_encoded_for_model,
    y_subset,
    eps=0.1,
    full_model_cols=X_train.columns,
    subset_model_cols=selected_features,
    p=0.075
)

print("Generated adversarial examples (problem space):", X_adv.shape)

#show which features affected by FGSM
diff = pd.DataFrame(X_adv, columns=X_train.columns) - X_test # Convert X_adv to DataFrame with correct columns for comparison
changed_features = (diff[numeric_features] != 0).sum(axis=0)
print(changed_features)


Shape of X_raw_subset_full: (22544, 41)
Shape of X_test_encoded_for_model: (22544, 9)
Shape of y_subset: (22544,)
Generated adversarial examples (problem space): (22544, 41)
num_compromised                  369
num_access_files                  72
num_outbound_cmds                  0
is_host_login                     11
srv_rerror_rate                 5648
srv_diff_host_rate              4975
dst_host_count                 22543
dst_host_srv_count             22543
dst_host_srv_diff_host_rate     6214
dtype: int64


In [15]:
# ===============================
#   PART 4 — Compute MANIFOLD SCORE
# ===============================

# PCA projection distance as manifold score
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances

pca = PCA(n_components=10)
pca.fit(X_train)

def manifold_score(x):
    proj = pca.inverse_transform(pca.transform(x))
    return np.mean((x - proj)**2, axis=1)

manifold_clean = manifold_score(X_train[:])
manifold_adv   = manifold_score(X_adv)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


In [16]:
# ===============================
#   PART 5 — Compute DB SCORE (Deep Boundary)
# ===============================

def db_score(model, x):
    # Reshape input for the LSTM model
    x_reshaped = np.expand_dims(x, axis=1)
    with tf.GradientTape() as tape:
        x_t = tf.convert_to_tensor(x_reshaped, dtype=tf.float32)
        tape.watch(x_t)
        pred = model(x_t)
    grad = tape.gradient(pred, x_t).numpy()
    # The gradient will also be 3D (batch, 1, features), so squeeze the middle dimension
    return np.mean(np.abs(grad[:, 0, :]), axis=1)

db_clean = db_score(ids, X_train[:])
db_adv   = db_score(ids, X_adv)



In [17]:
# ===============================
#   PART 6 — Build MANDA dataset
# ===============================

S1 = np.concatenate([manifold_clean, manifold_adv])
S2 = np.concatenate([db_clean, db_adv])
Y  = np.concatenate([np.zeros_like(manifold_clean), np.ones_like(manifold_adv)])

df_manda = pd.DataFrame({'manifold': S1, 'db': S2, 'label': Y})
df_manda.head()
#print(df_manda)


,manifold,db,label
0,0.112969,6.968190e-04,0.0
1,0.201916,3.264921e-04,0.0
2,0.028944,1.981406e-07,0.0
3,0.009108,5.778466e-25,0.0
4,0.009008,3.238309e-06,0.0


In [18]:
# ===============================
#   PART 7 — Train MANDA (Logistic Regression)
# ===============================

# clf = LogisticRegression()
# clf.fit(df_manda[['manifold', 'db']], df_manda['label'])

# pred = clf.predict(df_manda[['manifold', 'db']])
# print("MANDA accuracy:", accuracy_score(df_manda['label'], pred))
# print("AUC:", roc_auc_score(df_manda['label'], pred))

# ===============================
#   PART 7 — Train MANDA (LSTM)
# ===============================
# MANDA features
X_manda = df_manda[['manifold', 'db']].values
y_manda = df_manda['label'].values

# reshape for LSTM: (samples, timesteps=1, features=2)
X_manda_lstm = X_manda.reshape(X_manda.shape[0], 1, X_manda.shape[1])
manda_lstm=build_ids_lstm(X_manda_lstm.shape[2]) # Corrected: Use shape[2] for number of features
manda_lstm.fit(
    X_manda_lstm,
    y_manda,
    epochs=5,
    batch_size=256,
    validation_split=0.1,
    verbose=1
)

# predictions
y_pred_prob = manda_lstm.predict(X_manda_lstm).ravel()
y_pred = (y_pred_prob > 0.5).astype(int)

print("MANDA LSTM Accuracy:", accuracy_score(y_manda, y_pred))
print("MANDA LSTM AUC:", roc_auc_score(y_manda, y_pred_prob))


Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


523/523 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.9459 - loss: 0.3494 - val_accuracy: 0.6486 - val_loss: 1.0055
Epoch 2/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9609 - loss: 0.0975 - val_accuracy: 0.7025 - val_loss: 0.9403
Epoch 3/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9613 - loss: 0.0928 - val_accuracy: 0.6907 - val_loss: 0.9164
Epoch 4/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9618 - loss: 0.0882 - val_accuracy: 0.6998 - val_loss: 0.8750
Epoch 5/5
523/523 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9618 - loss: 0.0865 - val_accuracy: 0.6831 - val_loss: 0.8764
4642/4642 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step
MANDA LSTM Accuracy: 0.9337247587818229
MANDA LSTM AUC: 0.9793333051453673


In [ ]:
# ===============================
#   PART 8 — Fixing FPR at 5% or 15%
# ===============================
scores = clf.predict_proba(df_manda[['manifold', 'db']])[:,1]
fpr, tpr, th = roc_curve(df_manda['label'], scores)

def get_threshold(target_fpr):
    idx = np.argmin(np.abs(fpr - target_fpr))
    return th[idx]

thr_5  = get_threshold(0.05)
thr_15 = get_threshold(0.15)

print("Threshold at 5% FPR:", thr_5)
print("Threshold at 15% FPR:", thr_15)


Threshold at 5% FPR: 0.5452126028262122
Threshold at 15% FPR: 0.19321138608288213
